# Ejercicio: Crear un DataFrame Completo con SWAPI

## Objetivo:
El objetivo de este ejercicio es que los alumnos practiquen la obtención de datos de una API REST y su posterior procesamiento para crear un DataFrame en Pandas que contenga información detallada sobre personajes de Star Wars.

## Instrucciones:

1. Obtener los datos:

- Utilizar la API de Star Wars (SWAPI) para obtener datos sobre los personajes (people) de la saga.
- La URL base para obtener la lista de personajes es: https://swapi.dev/api/people/.
- Dado que los datos están paginados, deberán recorrer todas las páginas para obtener la lista completa de personajes.

2. Crear el DataFrame:

    Crear un DataFrame que incluya las siguientes columnas con la información detallada de cada personaje:

    <!-- - id: Identificador único del personaje.
    - name: Nombre del personaje.
    - height: Altura del personaje (en centímetros).
    - mass: Peso del personaje (en kilogramos).
    - hair_color: Color de cabello.
    - skin_color: Color de piel.
    - eye_color: Color de ojos.
    - birth_year: Año de nacimiento.
    - gender: Género.
    - homeworld: Nombre del planeta de origen (obtenido mediante una petición adicional a la API).
    - species: Nombre de la especie del personaje (obtenido mediante una petición adicional si el campo species no está vacío).
    - films: Lista de nombres de películas en las que aparece el personaje (obtenido mediante peticiones adicionales). -->

3. Procesamiento de datos:

    - Cada entrada en el campo homeworld de los personajes es un enlace a la API, desde el cual se debe obtener el nombre del planeta.

    - El campo species contiene una lista de enlaces. Si no está vacío, se debe obtener el nombre de la especie desde la API.

    - El campo films contiene una lista de enlaces a las películas. Se debe realizar una petición por cada enlace para obtener el título de la película.

4. Entrega del ejercicio:

    - Se debe entregar un archivo .ipynb (Jupyter Notebook) con el código que obtenga los datos, procese las respuestas y construya el DataFrame completo.

    - También deben incluir en el notebook una breve descripción de cómo organizaron el proceso para obtener los datos y construir el DataFrame.

Tips:

- Utilicen la librería requests para realizar las peticiones HTTP.
- Tengan en cuenta que la API utiliza paginación, por lo que deberán iterar sobre las páginas hasta obtener todos los datos.
- Los datos relacionados (homeworld, species, films) requieren realizar peticiones adicionales, así que gestionen adecuadamente los tiempos de espera (pueden considerar utilizar la función time.sleep() para evitar sobrecargar la API).
- Si encuentran campos con datos faltantes o errores, documenten cómo lo manejan en su notebook.

In [1]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed



In [9]:
a = requests.get('https://swapi.py4e.com/api/people/')
b = a.json()

In [10]:
def get_cararteres():
    url = "https://swapi.py4e.com/api/people/"
    all_people = []
    
    while url:
        res = requests.get(url)
        data = res.json()
        all_people.extend(data['results'])
        url = data['next']
    
    return all_people


caracteres = get_cararteres()

def find_name(url):
    try:
        res = requests.get(url)
        data = res.json()
        return data['name']
    except:
        return 'Error'
    
def find_film(films):
    lista_titles = []
    for f in films:
        try:
            res = requests.get(f)
            data = res.json()
            lista_titles.append(data['title'])  
        except:
            'Error'
            continue
    return lista_titles
    

In [4]:
swar_list = [{'name':swapi['name'],
              'height':swapi['height'],
              'mass':swapi['mass'],
              'hair_color':swapi['hair_color'],
              'skin_color':swapi['skin_color'],
              'eye_color':swapi['eye_color'],
              'birth_year':swapi['birth_year'],
              'gender':swapi['gender'],
              'homeworld':find_name(swapi['homeworld']),
              'species':find_name(swapi['species'][0]) if swapi['species'] else 'No tiene specie',
              'films':find_film(swapi['films'])}for swapi in caracteres]
df_swar = pd.DataFrame(swar_list)
df_swar['id'] = df_swar.index + 1
df_swar.set_index('id', inplace=True)

In [5]:
df_swar

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,species,films
id,,,,,,,,,,,
1,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,Tatooine,No tiene specie,"[A New Hope, The Empire Strikes Back, Return o..."
2,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,Tatooine,Droid,"[A New Hope, The Empire Strikes Back, Return o..."
3,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,Naboo,Droid,"[A New Hope, The Empire Strikes Back, Return o..."
4,Darth Vader,202,136,none,white,yellow,41.9BBY,male,Tatooine,No tiene specie,"[A New Hope, The Empire Strikes Back, Return o..."
5,Leia Organa,150,49,brown,light,brown,19BBY,female,Alderaan,No tiene specie,"[A New Hope, The Empire Strikes Back, Return o..."
...,...,...,...,...,...,...,...,...,...,...,...
78,Grievous,216,159,none,"brown, white","green, yellow",unknown,male,Kalee,Kaleesh,[Revenge of the Sith]
79,Tarfful,234,136,brown,brown,blue,unknown,male,Kashyyyk,Wookie,[Revenge of the Sith]
80,Raymus Antilles,188,79,brown,light,brown,unknown,male,Alderaan,No tiene specie,"[A New Hope, Revenge of the Sith]"
